In [2]:
%pip install -q "dlt[qdrant]" "qdrant-client[fastembed]"

/Users/ettyekhon/code/src/petroineos/petroineos-related/llm-zoomcamp-2025/.venv/bin/python3: No module named pip
Note: you may need to restart the kernel to use updated packages.


In [5]:
import requests
import dlt

In [9]:
print(dlt.__version__)

1.13.0


In [6]:
@dlt.resource
def zoomcamp_data():
    """
    Fetches document metadata from the Zoomcamp GitHub repository
    and yields one document record at a time, enriched with its course name.
    """
    docs_url = "https://github.com/alexeygrigorev/llm-rag-workshop/raw/main/notebooks/documents.json"
    docs_response = requests.get(docs_url)
    documents_raw = docs_response.json()

    for course in documents_raw:
        course_name = course["course"]
        for doc in course["documents"]:
            doc["course"] = course_name
            yield doc


In [10]:
from dlt.destinations import qdrant

qdrant_destination = qdrant(
    qd_path="db.qdrant",
)

In [11]:
pipeline = dlt.pipeline(
    pipeline_name="zoomcamp_pipeline",
    destination=qdrant_destination,
    dataset_name="zoomcamp_tagged_data",
)
load_info = pipeline.run(zoomcamp_data())
print(pipeline.last_trace)


Fetching 5 files:   0%|          | 0/5 [00:00<?, ?it/s]

tokenizer.json: 0.00B [00:00, ?B/s]

config.json:   0%|          | 0.00/701 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/366 [00:00<?, ?B/s]

model_optimized.onnx:   0%|          | 0.00/133M [00:00<?, ?B/s]

Run started at 2025-07-11 21:58:14.206283+00:00 and COMPLETED in 2 minutes and 21.83 seconds with 4 steps.
Step extract COMPLETED in 1.15 seconds.

Load package 1752271231.8282309 is EXTRACTED and NOT YET LOADED to the destination and contains no failed jobs

Step normalize COMPLETED in 0.05 seconds.
Normalized data for the following tables:
- _dlt_pipeline_state: 1 row(s)
- zoomcamp_data: 948 row(s)

Load package 1752271231.8282309 is NORMALIZED and NOT YET LOADED to the destination and contains no failed jobs

Step load COMPLETED in 3.00 seconds.
Pipeline zoomcamp_pipeline load step completed in 3.00 seconds
1 load package(s) were loaded to destination qdrant and into dataset zoomcamp_tagged_data
The qdrant destination used /Users/ettyekhon/code/src/petroineos/petroineos-related/llm-zoomcamp-2025/packages/vector_search/src/vector_search/notebooks/db.qdrant location to store data
Load package 1752271231.8282309 is LOADED and contains no failed jobs

Step run COMPLETED in 2 minutes and